# Build ETL pipeline on AWS Redshift Data Warehouse

In [3]:
%load_ext sql

In [4]:
from time import time
import boto3
import json
import pandas as pd

<hr>

# Part 1 - Create AWS Redshift Cluster
- Use Python SDK boto3 to create new AWS Redshift cluster

## Load AWS Redshift Parameters from configuration file

In [5]:
import configparser

config = configparser.ConfigParser()
config.read_file(open('dwh.cfg'))

KEY                    = config.get('AWS','KEY')
SECRET                 = config.get('AWS','SECRET')

cluster_type       = config.get("CLUSTER","cluster_type")
num_nodes          = config.get("CLUSTER","num_nodes")
node_type          = config.get("CLUSTER","node_type")
cluster_identifier = config.get("CLUSTER","cluster_identifier")
dbname                 = config.get("CLUSTER","dbname")
user            = config.get("CLUSTER","user")
password        = config.get("CLUSTER","password")
port               = config.get("CLUSTER","port")
iam_role_name = config.get("CLUSTER", "iam_role_name")

pd.DataFrame({"Param":
                  ["cluster_type", "num_nodes", "node_type", "cluster_identifier", "dbname", "user", "password", "port", "iam_role_name"],
              "Value":
                  [cluster_type, num_nodes, node_type, cluster_identifier, dbname, user, password, port, iam_role_name]
             })

,Param,Value
0,cluster_type,multi-node
1,num_nodes,4
2,node_type,dc2.large
3,cluster_identifier,dwhCluster
4,dbname,sparkify
5,user,dwhadmin
6,password,Password1!
7,port,5439
8,iam_role_name,dwhRole


## Create IAM and Redshift client

In [6]:
iam = boto3.client('iam',aws_access_key_id=KEY,
                     aws_secret_access_key=SECRET,
                     region_name='us-west-2'
                  )

In [7]:
redshift = boto3.client('redshift',
                       region_name="us-west-2",
                       aws_access_key_id=KEY,
                       aws_secret_access_key=SECRET
                       )

## Create Redshift Cluster

In [8]:
#GET Role ARN
roleArn = iam.get_role(RoleName=iam_role_name)['Role']['Arn']

print(roleArn)

arn:aws:iam::273070980972:role/dwhRole


In [9]:
def create_redshift_cluster():
    """
    Create new redshift cluster database
    """
    
    try:
        response = redshift.create_cluster(        
            #Hardware
            ClusterType=cluster_type,
            NodeType=node_type,
            NumberOfNodes=int(num_nodes),

            #Identifiers & Credentials
            DBName=dbname,
            ClusterIdentifier=cluster_identifier,
            MasterUsername=user,
            MasterUserPassword=password,

            #Roles (for s3 access)
            IamRoles=[roleArn]  
        )
    except Exception as e:
        print(e)

In [10]:
create_redshift_cluster()

## Display the cluster status

In [12]:
def display_redshift_status(props):
    pd.set_option('display.max_colwidth', -1)
    keysToShow = ["ClusterIdentifier", "NodeType", "ClusterStatus", "MasterUsername", "DBName", "Endpoint", "NumberOfNodes", 'VpcId']
    x = [(k, v) for k,v in props.items() if k in keysToShow]
    return pd.DataFrame(data=x, columns=["Key", "Value"])

myClusterProps = redshift.describe_clusters(ClusterIdentifier=cluster_identifier)['Clusters'][0]
display_redshift_status(myClusterProps)

,Key,Value
0,ClusterIdentifier,dwhcluster
1,NodeType,dc2.large
2,ClusterStatus,creating
3,MasterUsername,dwhadmin
4,DBName,sparkify
5,VpcId,vpc-c3085cbb
6,NumberOfNodes,4


## Get the host of the created redshift cluster
 - The redshift cluster <font color='red'>endpoint</font>

In [13]:
DWH_ENDPOINT='dwhcluster.cyihktxlk33b.us-west-2.redshift.amazonaws.com'

<hr>

# Part 2 - Create Table Schemas
`Instructions`
1. Open Terminal window
2. At terminal prompt, type: python create_tables.py

<strong>create_tables.py</strong>
- Create database schema for new tables
- Drop tables if any exist
- Create tables

<hr>

# Part 3 - Build ETL Pipeline
`Instructions`
1. At terminal prompt, type: python etl.py

<strong>etl.py</strong>
- Load staging tables for event and song data from from AWS S3 json files
- Load Data Warehouse dimension and fact tables from staging tables